*written by: Julia Knoblauch, Jana Vihs, Annika Boer, Kai Ingo Schewina*

# Explainable Recommendation Systems for Energy-Efficient Smart Home

requirements.txt

## Table of contents

1. Abstract
2. Introduction
3. Literature Review
    - Explainable AI (XAI)
    - Recommender Systems in SH
4. Current Recommender System --> agents + Bild (Jana)
5. Methodology (i.e., the ML techniques that you apply)
    - Recommender System??
    - Algorithms XAI
        - Taxonomy (Bild) (Annika)
        - Model-intrinsic (Annika)
        - Model-agnostic (Machine Learning Interpretability: A Survey on Methods and Metrics)
            - LIME (Annika)
            - SHAP (Julia)
            - other approaches?
        - vorgestellte Tabelle aus Präsi formaler --> Vergleich
    - Algorithms prediction (Jana, Formel für finales Modell, mit paper)
        - xgboost
        - adaboost
        - knn
        - logit
        - rf
        - glm?
6. Experimental Design (including EDA, data organization, performance measures, etc.)
    - data organization (+ EDA von denen)
        - agents system (Bild) (Jana)
        - restriction to 2 households
        - with & without weather data (EDA, Struktur, Herkunft) (Kai)
    - Evaluation
        - Performance (Kai?)
        - Explainability (feature importance) (Julia, Annika)
7. Results
    - Performance (Kai?)
    - Explainability (Julia, Annika)
    - Decision for model & XAI model
    - Recommendation + Explanation (Jana)
8. Discussion and Limitations
    - no user
    - runtime problems
        - api weather data 
        - 4 models, 20 households
        - Beispiel, wie lange es dauern würde, eine recommendation zu laden (aber nur 1x am Tag)
9. References